<a href="https://colab.research.google.com/github/Ape108/Quantum_Portfolio_Optimization/blob/main/MVP_Completed_Spiced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Optimization using QAOA

This notebook demonstrates how to solve a cardinality-constrained portfolio optimization problem using the Quantum Approximate Optimization Algorithm (QAOA).

The process involves these key steps:
1.  **Data Acquisition**: Fetching the top holdings of a target ETF (XLK) and their historical price data.
2.  **Financial Modeling**: Calculating the expected annual returns (`mu`) and the covariance matrix (`Sigma`) for the assets.
3.  **QUBO Formulation**: Translating the financial problem (maximize return, minimize risk) into a Quadratic Unconstrained Binary Optimization (QUBO) matrix `Q`. This includes a penalty term to enforce the constraint on the number of assets to select.
4.  **Quantum Solution**: Using Qiskit's QAOA implementation to find the bitstring that minimizes the QUBO, which corresponds to the optimal portfolio selection.
5.  **Analysis**: Running the QAOA with an increasing number of repetitions (`reps`) to analyze how the solution quality improves.

## 1. Setup and Environment

This section handles the initial setup, including installing necessary libraries and importing the required modules from standard Python, third-party data science libraries, and Qiskit.

In [ ]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00


In [ ]:
# ---- Standard library ----
import os
import re
import io
import time
import subprocess
import shlex

# ---- Third-party libraries ----
import numpy as np
import pandas as pd
import requests
import yfinance as yf
from collections import defaultdict

# ---- Quantum Circuit ----
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.circuit.library import QAOAAnsatz
from qiskit.visualization import plot_histogram

# ---- Colab/Google ----
from google.colab import drive

# ---- Warnings ----
import warnings
from scipy.sparse import SparseEfficiencyWarning

# This will ignore all SparseEfficiencyWarning messages
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)


## 2. Data Acquisition and Financial Modeling

Here, we define our target ETF and fetch the necessary financial data.

- **Fetch Holdings**: We start by scraping the top 5 holdings for the Technology Select Sector SPDR Fund (XLK) from the SSGA website.
- **Download Price History**: Using `yfinance`, we download 1-minute interval price data for the last 7 days for these tickers.
- **Calculate Returns and Covariance**: From the price data, we compute the percentage change to get minute-by-minute returns. These are then annualized to produce our key financial inputs:
    - `mu`: The vector of expected annual returns for each asset.
    - `Sigma`: The annualized covariance matrix, which represents the risk and correlation between the assets.

In [ ]:
def fetch_holdings_from_ssga(symbol: str) -> pd.DataFrame:
    """
    Return a clean DataFrame of holdings for an SSGA ETF.
    This is a simplified version that assumes a predictable Excel format.
    """
    # 1. Download the Excel file into an in-memory buffer
    url = f"https://www.ssga.com/library-content/products/fund-data/etfs/us/holdings-daily-us-en-{symbol.lower()}.xlsx"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    excel_file = io.BytesIO(response.content)

    # 2. Find the header row by searching for 'Ticker' or 'Symbol'
    # We read the first 40 rows just to inspect them
    temp_df = pd.read_excel(excel_file, header=None, dtype=str, nrows=40)
    header_row_index = None
    for i, row in temp_df.iterrows():
        # Check if any cell in the row contains 'Ticker' or 'Symbol' (case-insensitive)
        if row.str.contains(r'^(ticker|symbol)$', case=False, na=False).any():
            header_row_index = i
            break

    if header_row_index is None:
        raise RuntimeError("Could not find the header row in the Excel file.")

    # 3. Read the Excel file again, this time with the correct header
    # The buffer automatically resets, so we can read from it again
    df = pd.read_excel(excel_file, header=header_row_index)

    # 4. Find the correct column for the tickers
    ticker_col_candidates = ['ticker', 'symbol', 'ticker symbol']
    ticker_col = next((col for col in df.columns if str(col).strip().lower() in ticker_col_candidates), None)

    if ticker_col is None:
        raise RuntimeError(f"Could not find a ticker column. Found columns: {list(df.columns)}")

    # 5. Extract, clean, and format the ticker data into a final DataFrame
    tickers = df[ticker_col].dropna().astype(str)

    # Filter for strings that look like valid tickers
    valid_ticker_mask = tickers.str.match(r"^[A-Z0-9]{1,6}([.-][A-Z0-9]{1,3})?$", case=False)

    clean_tickers = (
        tickers[valid_ticker_mask]
        .str.upper()
        .str.replace("-", ".", regex=False) # Standardize tickers like 'BRK-B' to 'BRK.B'
        .drop_duplicates()
    )

    return pd.DataFrame({"Ticker": clean_tickers}).reset_index(drop=True)




In [ ]:
PRICE_COLS = ["Open","High","Low","Close","Adj Close","Volume"]
ETF_SYMBOL = "XLK"
tickers = fetch_holdings_from_ssga(ETF_SYMBOL).head(5) # Grab 5 tickers
tickers.head(5)

/tmp/ipython-input-961546454.py:18: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if row.str.contains(r'^(ticker|symbol)$', case=False, na=False).any():


,Ticker
0,NVDA
1,MSFT
2,AAPL
3,AVGO
4,ORCL


## Data Fetch and Feature Setup

This cell:
- Cleans `tickers`.
- Downloads 1-minute prices (7 days), builds `close_df`, computes returns, annualized `mu` and `Sigma`.
- Fetches fundamentals per ticker: `priceToSalesTTM`, `profitMargin`, `debtToEquity`, `dividendYield`, and `revenueGrowth`.
- Prints snapshots and an optional `fundamentals_view` (last price + fundamentals).

Outputs:
- `close_df`, `minute_returns`, `mu`, `Sigma`
- `fund_df` with P/S (TTM), profit margin, D/E, dividend yield, revenue growth
- `fundamentals_view` (optional)

Notes:
- Dividends: use `dividendYield` for cross-company comparison.
- Growth: use `revenueGrowth` as the primary growth feature.


In [ ]:
# --- Make sure tickers is a list of strings ---
if isinstance(tickers, pd.DataFrame):
    tickers = tickers.squeeze()
if isinstance(tickers, pd.Series):
    tickers = tickers.dropna().astype(str).unique().tolist()
elif isinstance(tickers, str):
    tickers = [tickers]
else:
    tickers = list(map(str, tickers))

# --- Download 1-minute data for the last 7 days ---
data = yf.download(
    tickers,
    period="7d",
    interval="1m",
    auto_adjust=True,
    group_by="ticker",
    threads=True
)

# --- Build a Close-price DataFrame ---
if len(tickers) == 1:
    close_df = data[['Close']].rename(columns={'Close': tickers[0]})
else:
    pieces = {}
    for t in tickers:
        if (t in data) and ('Close' in data[t].columns):
            pieces[t] = data[t]['Close']
    if not pieces:
        raise RuntimeError("No Close data retrieved for any tickers.")
    close_df = pd.concat(pieces, axis=1)

# --- 1-minute simple returns ---
minute_returns = close_df.pct_change().dropna(how="all")
print("\n--- Sample of 1-minute returns ---")
print(minute_returns.head())

# --- Annualization (per-minute to per-year) ---
MIN_PER_DAY = 390
TRADING_DAYS = 252
MIN_PER_YEAR = MIN_PER_DAY * TRADING_DAYS

mu = minute_returns.mean() * MIN_PER_YEAR
Sigma = minute_returns.cov() * MIN_PER_YEAR

print("\n--- mu (annualized) ---")
print(mu.head() if hasattr(mu, "head") else mu)
print("\n--- Sigma (annualized, top-left 5x5) ---")
print(Sigma.iloc[:5, :5])

# =========================
# Fundamentals: P/S (TTM), Profit Margin, Debt-to-Equity, Dividend Yield, Revenue Growth
# =========================
def _safe_get_info(tkr: yf.Ticker) -> dict:
    try:
        d = tkr.get_info()
        return d if isinstance(d, dict) else {}
    except Exception:
        try:
            d = tkr.info
            return d if isinstance(d, dict) else {}
        except Exception:
            return {}

def fetch_fundamentals(tickers_list):
    rows = []
    for t in tickers_list:
        info = _safe_get_info(yf.Ticker(t))
        rows.append({
            "Ticker": t,
            "priceToSalesTTM": info.get("priceToSalesTrailing12Months"),
            "profitMargin": info.get("profitMargins"),   # decimal (e.g., 0.18 == 18%)
            "debtToEquity": info.get("debtToEquity"),
            "dividendYield": info.get("dividendYield"),  # decimal (e.g., 0.02 == 2%)
            "revenueGrowth": info.get("revenueGrowth"),  # decimal YoY
        })
    return pd.DataFrame(rows).set_index("Ticker")

fund_df = fetch_fundamentals(tickers)
print("\n--- Fundamentals (P/S, Profit Margin, D/E, Dividend Yield, Growth) ---")
print(fund_df)

# --- Optional: join with last price for readability ---
fundamentals_view = None
try:
    last_prices = close_df.iloc[-1].rename("LastPrice").to_frame()
    fund_view_tmp = fund_df.copy()
    for c in ["profitMargin", "dividendYield", "revenueGrowth"]:
        if c in fund_view_tmp:
            fund_view_tmp[f"{c}(%)"] = (fund_view_tmp[c] * 100).round(2)
    fundamentals_view = last_prices.join(
        fund_view_tmp[
            ["priceToSalesTTM",
             "profitMargin", "profitMargin(%)",
             "debtToEquity",
             "dividendYield", "dividendYield(%)",
             "revenueGrowth", "revenueGrowth(%)"]
        ],
        how="left"
    )
    print("\n--- Last price + key fundamentals ---")
    print(fundamentals_view)
except Exception as e:
    print("\n(Info) Could not join last prices with fundamentals:", e)


[*********************100%***********************]  5 of 5 completed
/tmp/ipython-input-1621137811.py:34: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  minute_returns = close_df.pct_change().dropna(how="all")



--- Sample of 1-minute returns ---
                               NVDA      MSFT      AAPL      AVGO      ORCL
Datetime                                                                   
2025-09-29 13:31:00+00:00  0.004133  0.000354 -0.001476 -0.000294 -0.001898
2025-09-29 13:32:00+00:00 -0.000768  0.001102  0.002464  0.001551 -0.002078
2025-09-29 13:33:00+00:00  0.002963 -0.000859 -0.000708  0.001590  0.004783
2025-09-29 13:34:00+00:00 -0.001366 -0.000127 -0.001889  0.000849  0.000386
2025-09-29 13:35:00+00:00  0.001804  0.000576 -0.000946  0.001827  0.000370

--- mu (annualized) ---
NVDA    0.727518
MSFT    0.878482
AAPL    0.359094
AVGO    0.002261
ORCL    0.125974
dtype: float64

--- Sigma (annualized, top-left 5x5) ---
          NVDA      MSFT      AAPL      AVGO      ORCL
NVDA  0.079128  0.003812  0.005582  0.052294  0.023699
MSFT  0.003812  0.027003  0.001410  0.005130  0.016614
AAPL  0.005582  0.001410  0.029726  0.007040 -0.000476
AVGO  0.052294  0.005130  0.007040  0.131245 

## 3. QUBO Formulation

To solve this problem on a quantum computer, we must formulate it as a QUBO. The goal is to find a binary vector `x` where `x_i = 1` if we select the asset and `0` otherwise.

The objective function to be minimized is:

$ \min_{x \in \{0,1\}^n} \left( x^T \Sigma x - q \mu^T x \right) + P \left( \sum_{i=1}^n x_i - K \right)^2 $

This function consists of three parts:
1.  **Risk Term ($x^T \Sigma x$)**: The portfolio variance, which we want to minimize.
2.  **Return Term ($-q \mu^T x$)**: The negated expected return. The parameter `q` balances the trade-off between maximizing return and minimizing risk.
3.  **Constraint Penalty ($P (\sum x_i - K)^2$)**: A penalty term that ensures the total number of selected assets is exactly equal to our target `K`. The penalty coefficient `P` must be large enough to make violating the constraint undesirable for the optimizer.

The functions below build the corresponding QUBO matrix `Q` from `mu`, `Sigma`, `K`, `q`, and `P`.

In [ ]:

def build_qubo_Q(tickers, Sigma: pd.DataFrame, mu: pd.Series, K: int = 3, q: float = 0.5, P: float = 10.0):
    """
    Build a QUBO Q matrix for cardinality-constrained portfolio selection.

    Args:
        tickers: list-like of tickers, defines variable order (x_i for tickers[i])
        Sigma: covariance matrix (pd.DataFrame) indexed/columned by tickers
        mu: expected returns (pd.Series) indexed by tickers
        K: number of assets to select
        q: risk/return tradeoff (higher puts more weight on returns term)
        P: penalty weight for enforcing sum(x_i)=K

    Returns:
        Q: defaultdict(float) with keys (i,j) in 0..n-1
        index_of: dict mapping ticker -> index
    """
    tickers = list(tickers)
    num_assets = len(tickers)
    index_of = {t: i for i, t in enumerate(tickers)}

    # align Sigma and mu to the given tickers order
    Sigma_aligned = Sigma.loc[tickers, tickers]
    mu_aligned = mu.loc[tickers]

    Q = defaultdict(float)

    # 1) Risk term (covariance)
    for i in range(num_assets):
        for j in range(num_assets):
            Q[(i, j)] += float(Sigma_aligned.iloc[i, j])

    # 2) Return term (negated for minimization)
    for i in range(num_assets):
        Q[(i, i)] -= float(q) * float(mu_aligned.iloc[i])

    # 3) Constraint penalty: (sum x_i - K)^2 = (1 - 2K)*sum x_i + 2*sum_{i<j} x_i x_j + const
    for i in range(num_assets):
        Q[(i, i)] += float(P) * (1 - 2 * K)
        for j in range(i + 1, num_assets):
            Q[(i, j)] += float(P) * 2.0

    return Q, index_of

# ---------- Example usage (mirrors your snippet) ----------
# # user parameters
# K = 3
# q = 0.5
# P = 1.0
#
# tickers = xlk_top_ten  # your list
# Q, index_of = build_qubo_Q(tickers, Sigma=Sigma, mu=mu, K=K, q=q, P=P)

In [ ]:
def qubo_to_matrix_df(Q_dict, tickers):
    n = len(tickers)
    M = np.zeros((n, n), dtype=float)
    for (i, j), v in Q_dict.items():
        M[i, j] += float(v)
    # symmetrize (xᵀQx expects a symmetric Q)
    M = (M + M.T) / 2.0
    return pd.DataFrame(M, index=tickers, columns=tickers)

def normalize_qubo_df(Q_df, limit=1.0):
    """Scale Q so that max |entry| == limit. Returns (Q_scaled, scale)."""
    m = Q_df.abs().to_numpy().max()
    if m == 0:
        return Q_df.copy(), 1.0
    s = limit / m
    return Q_df * s, s


In [ ]:
K = 2
q = 0.5
P = 10.0

Q_dict, index_of = build_qubo_Q(tickers, Sigma, mu, K=K, q=q, P=P)
Q_df = qubo_to_matrix_df(Q_dict, tickers)

Q_norm, _ = normalize_qubo_df(Q_df)

Q_norm  # <- this is your matrix version of Q


,NVDA,MSFT,AAPL,AVGO,ORCL
NVDA,-0.995804,0.328940,0.328999,0.330535,0.329594
MSFT,0.328940,-1.000000,0.328861,0.328984,0.329361
AAPL,0.328999,0.328861,-0.991371,0.329046,0.328799
AVGO,0.330535,0.328984,0.329046,-0.982167,0.330185
ORCL,0.329594,0.329361,0.328799,0.330185,-0.982141


## 4. Solving with QAOA

The QUBO problem is solved using QAOA. This involves a few helper functions to manage the conversion and execution.

- **QUBO to Ising Hamiltonian**: The `qubo_to_ising_sparsepauli` function converts the QUBO matrix `Q` into an equivalent Ising Hamiltonian `H` and an energy offset. QAOA works by finding the ground state (minimum energy) of this Hamiltonian.
- **QAOA Driver**: The `run_qaoa_qubo` function orchestrates the entire process. It takes the QUBO matrix, creates a `QAOAAnsatz` circuit, and uses a classical optimizer (`L-BFGS-B`) to find the optimal parameters (`gamma` and `beta`) that prepare the ground state.
- **Statevector Simulation**: For stability and accuracy, the objective function is evaluated using a noiseless statevector simulator. The final result is determined by finding the most probable bitstring in the final optimized statevector.

In [ ]:
def ensure_Q_df(Q_df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep ONLY diagonal + i<j (upper triangle). Do NOT average with transpose.
    Also force real float and remove NaN/Inf.
    """
    A = Q_df.to_numpy(copy=True)
    A = np.real_if_close(A, tol=1e5)
    if np.iscomplexobj(A):
        A = A.real
    A = np.nan_to_num(A, nan=0.0, posinf=0.0, neginf=0.0)
    A = np.triu(A, 0)  # keep diag and i<j; zero lower triangle
    return pd.DataFrame(A, index=Q_df.index, columns=Q_df.columns)

def qubo_to_ising_sparsepauli(Q_df: pd.DataFrame):
    """
    Map QUBO min x^T Q x (x in {0,1}^n) to Ising with x_i = (1 - z_i)/2, z_i ∈ {-1,+1}.
    Assumes/forces Q to be upper-triangle-only (diag + i<j once).
    Returns (SparsePauliOp H, energy_offset).
    """
    Qu = ensure_Q_df(Q_df)                  # enforce upper-triangle + real
    Q = Qu.to_numpy(copy=True)
    n = Q.shape[0]

    h = np.zeros(n, dtype=float)
    offset = 0.0
    labels, coeffs = [], []

    # Off-diagonal pair terms: for each i<j with qij
    # x_i x_j = (1 - z_i - z_j + z_i z_j)/4
    for i in range(n):
        for j in range(i + 1, n):
            qij = Q[i, j]
            if qij == 0.0:
                continue
            offset += 0.25 * qij
            h[i]    += -0.25 * qij
            h[j]    += -0.25 * qij
            Jij = 0.25 * qij
            # add ZZ term (Qiskit uses little-endian Pauli strings)
            z = ["I"] * n
            z[n - 1 - i] = "Z"
            z[n - 1 - j] = "Z"
            labels.append("".join(z))
            coeffs.append(float(Jij))

    # Diagonal (linear) terms: Q_ii * x_i = Q_ii * (1 - z_i)/2
    for i in range(n):
        qii = Q[i, i]
        if qii != 0.0:
            offset += 0.5 * qii
            h[i]    += -0.5 * qii

    # Single-Z terms
    for i, hi in enumerate(h):
        if hi != 0.0:
            z = ["I"] * n
            z[n - 1 - i] = "Z"
            labels.append("".join(z))
            coeffs.append(float(hi))

    H = SparsePauliOp.from_list(list(zip(labels, coeffs))) if labels else SparsePauliOp.from_list([("I" * n, 0.0)])

    # Final guard: ensure strictly real coeffs
    if np.iscomplexobj(H.coeffs):
        H = SparsePauliOp(H.paulis, coeffs=np.real(H.coeffs))

    return H, float(offset)

def bitstring_from_statevector_probs(probs_dict):
    """Return the most likely bitstring and its probability from a probs dict."""
    if not probs_dict:
        return None, 0.0
    items = sorted(probs_dict.items(), key=lambda kv: kv[1], reverse=True)
    return items[0][0], items[0][1]


In [ ]:
def run_qaoa_qubo(Q_df, reps=2, shots=4096, seed=42):
    """
    QAOA on a QUBO given as DataFrame Q_df.
    Assumes Q_df was already cleaned/upper-triangle elsewhere.
    This version guarantees a real cost Hamiltonian.
    """
    rng = np.random.default_rng(seed)

    # QUBO -> Ising
    H, offset = qubo_to_ising_sparsepauli(Q_df)

    # --- enforce strictly REAL coefficients (fix for "complex coefficients" error) ---
    # Strip any tiny imag parts that may appear from numeric noise
    from qiskit.quantum_info import SparsePauliOp
    if np.iscomplexobj(H.coeffs):
        H = SparsePauliOp(H.paulis, coeffs=np.real(H.coeffs))
    offset = float(np.real_if_close(offset))

    # QAOA ansatz (default X-mixer)
    ansatz = QAOAAnsatz(cost_operator=H, reps=reps)

    # Parameters
    param_vec = ansatz.parameters
    d = len(param_vec)
    x0 = rng.uniform(-np.pi, np.pi, size=d)

    # Objective: Re(<H>) + offset  (guard with real_if_close)
    def objective(theta):
        bound = ansatz.assign_parameters(dict(zip(param_vec, theta)), inplace=False)
        psi = Statevector.from_instruction(bound)
        val = psi.expectation_value(H)
        return float(np.real_if_close(val) + offset)

    # Optimize
    bounds = [(-np.pi, np.pi)] * d
    res = minimize(objective, x0, method="L-BFGS-B", bounds=bounds, options={"maxiter": 300})
    best_val = float(res.fun)

    # Final state & measurement probs
    final_circ = ansatz.assign_parameters(dict(zip(param_vec, res.x)), inplace=False)
    sv = Statevector.from_instruction(final_circ)
    probs = sv.probabilities_dict()

    best_bits, best_prob = bitstring_from_statevector_probs(probs)

    # Optional: plot
    # plot_histogram(probs, title="QAOA Bitstring Distribution (statevector)"); plt.show()

    return {
        "energy": best_val,
        "bits": best_bits,
        "prob": best_prob,
        "probs": probs,
        "params": res.x,
        "tickers": list(Q_df.index),
    }


## 5. Experiment and Analysis

Finally, we run the experiment.

We print:
- The approximate ground energy found by the algorithm.
- The most probable bitstring solution (`1` indicates a selected asset).
- The list of ticker symbols corresponding to the optimal portfolio.

In [ ]:
num_reps = 1

result = run_qaoa_qubo(Q_df, reps=num_reps, shots=4096, seed=42)
print("Approx. ground energy (incl. offset):", result["energy"])
print("Most probable bitstring:", result["bits"], f"(p={result['prob']:.3f})")
print("Selected tickers:", [t for t, b in zip(result["tickers"], result["bits"][::-1]) if b == "1"])



Approx. ground energy (incl. offset): -57.12670098370214
Most probable bitstring: 00111 (p=0.341)
Selected tickers: ['NVDA', 'MSFT', 'AAPL']
